In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=b05ee9a91e0d5f130b479393e725c09b31874f39b54eda1e9aeb8c0d8b58af91
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
from pyspark import SparkContext

Create a Spark context

In [3]:
sc = SparkContext()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/data.txt"
# Read the text file into an RDD
lines = sc.textFile(path)

Mounted at /content/drive


I use two-step map-reduce for this problem.<br/>
First step:

In [5]:
# Map each line to a list of channel IDs
channel_lists = lines.map(lambda line: line.split())

# Map each channel pair to a key-value pair, where the key is the pair and the value is the count
channel_pairs = channel_lists.flatMap(lambda channel_list: [(tuple(sorted((channel_list[0], channel))), 1) for channel in channel_list[1:]])

# Reduce the key-value pairs by summing up the counts for each pair
total_counts = channel_pairs.reduceByKey(lambda count1, count2: ((count1 + count2)/2))

Secound step:

In [6]:
# Map each channel pair to a key-value pair, where the key is a channel and the value is the total count
channel_counts = total_counts.flatMap(lambda pair: [(pair[0][0], pair[1]), (pair[0][1], pair[1])])

# Reduce the key-value pairs by summing up the counts for each channel
total_channel_counts = channel_counts.reduceByKey(lambda count1, count2: count1 + count2)

In this part, I show 5 best links for link exchange


In [7]:
# Sort the key-value pairs by count in descending order
sorted_counts = total_channel_counts.sortBy(lambda pair: pair[1], ascending=False)

# Take the top five key-value pairs and output the channel IDs with the highest counts
top_channels = sorted_counts.take(5)
for channel, count in top_channels:
    print(f'Channel {channel} has been advertised {count} times.')

Channel 859 has been advertised 1933.0 times.
Channel 5306 has been advertised 1741.0 times.
Channel 2664 has been advertised 1528.0 times.
Channel 5716 has been advertised 1426.0 times.
Channel 6306 has been advertised 1394.0 times.


Filter for the key-value pair where the key is 1748

In [8]:
filtered_counts = total_counts.filter(lambda pair: "1748" in pair[0])

# Sum up the counts for the filtered pair
count_1748 = filtered_counts.map(lambda pair: pair[1]).sum()
count_1748

130.0


Filter for the key-value pair where the key is 5633

In [9]:
filtered_counts = total_counts.filter(lambda pair: "5633" in pair[0])

# Sum up the counts for the filtered pair
count_5633 = filtered_counts.map(lambda pair: pair[1]).sum()
count_5633

30.0

Filter for the key-value pair where the key is 3469

In [10]:
filtered_counts = total_counts.filter(lambda pair: "3469" in pair[0])

# Sum up the counts for the filtered pair
count_3469 = filtered_counts.map(lambda pair: pair[1]).sum()
count_3469

119.0